In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

dataPath = '/content/drive/MyDrive/Soccer Player Injury Prediction/Datasets/Final Dataset/injuryFinalV8582.csv'
df = pd.read_csv(dataPath)
df

,daily_load,fatigue,mood,readiness,sleep_duration,sleep_quality,soreness,stress,injury_ts,weekly_load,...,illness,historical_injury_count,stress_readiness_interaction,team_performance_readiness_interaction,offensive_performance_readiness_interaction,defensive_performance_readiness_interaction,acwr_rolling_avg,stress_rolling_sum,injury,Player_name
0,0.0,3.0,4.0,6.0,8.0,4.0,3.0,0.75,0.0,0.000000,...,0,0,24.0,38.16,41.64,36.84,0.320870,10.50,0,TeamA-3e5f6e2b-46b7-4890-84a9-3bbb2649af5a
1,0.0,3.0,3.0,6.0,7.0,3.0,3.0,0.50,0.0,0.000000,...,0,0,18.0,33.48,30.42,29.88,0.286085,7.00,0,TeamB-4405bb1f-56f7-48ba-bfa8-e795e4006952
2,0.0,3.0,3.0,8.0,8.0,3.0,3.0,0.50,0.0,0.000000,...,0,0,24.0,56.56,56.48,56.72,0.325813,7.00,0,TeamB-48bfd4ea-e9e2-45b5-befc-3383cae87fdf
3,0.0,3.0,3.0,8.0,8.0,3.0,3.0,0.50,0.0,0.000000,...,0,0,24.0,43.36,44.88,51.20,0.272225,7.00,0,TeamB-2f23d7d5-2326-49ce-b9c8-5a6303f785c5
4,200.0,4.0,3.0,8.0,8.0,4.0,3.0,0.50,0.0,0.488536,...,0,0,24.0,56.40,53.92,53.20,0.260681,7.25,0,TeamA-af719df9-3e6c-4ad4-9e8e-0c0c45f4f76a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8577,320.0,2.0,3.0,5.0,7.0,4.0,2.0,0.50,0.0,0.440917,...,0,0,15.0,32.40,27.50,29.15,0.175310,7.00,0,TeamA-4051bba7-1170-4c43-b912-8c38815a7625
8578,0.0,3.0,4.0,6.0,8.0,3.0,2.0,0.25,0.0,0.176367,...,0,0,12.0,40.74,27.60,34.38,0.246456,3.50,0,TeamA-f20565cc-df22-46a7-aa97-af8ed00601d7
8579,0.0,3.0,3.0,7.0,8.0,4.0,3.0,0.50,0.0,0.000000,...,0,0,21.0,46.90,45.57,46.62,0.244719,7.00,0,TeamA-e920ae60-5c4b-4597-be27-fc6876dcec33
8580,0.0,3.0,3.0,8.0,8.0,3.0,3.0,0.50,0.0,0.000000,...,0,0,24.0,57.36,60.96,55.84,0.288546,7.00,0,TeamB-2f23d7d5-2326-49ce-b9c8-5a6303f785c5


In [ ]:
# Print all column names
print("Column Names:")
print(df.columns)

Column Names:
Index(['daily_load', 'fatigue', 'mood', 'readiness', 'sleep_duration',
       'sleep_quality', 'soreness', 'stress', 'injury_ts', 'weekly_load',
       'Team', 'Total_distance', 'Average_running_speed', 'Top_speed', 'HIR',
       'acwr', 'atl', 'ctl28', 'monotony', 'strain', 'team_performance',
       'offensive_performance', 'defensive_performance', 'illness',
       'historical_injury_count', 'stress_readiness_interaction',
       'team_performance_readiness_interaction',
       'offensive_performance_readiness_interaction',
       'defensive_performance_readiness_interaction', 'acwr_rolling_avg',
       'stress_rolling_sum', 'injury', 'Player_name'],
      dtype='object')


In [ ]:
#Defining class imbalce functions

#Oversampling
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

def oversample_data(X_train, y_train, sampling_ratio=0.35):

    # Apply oversampling on the training data
    sampling_ratio = 0.35
    ros = RandomOverSampler(sampling_strategy=sampling_ratio, random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after oversampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100
    #print(f"Percentage of 0s after oversampling: {zero_percentage_after:.2f}%")

    return X_train_resampled, y_resampled, zero_percentage_after


# Undersampling
def undersample_data(X_train, y_train, sampling_ratio=0.35):

    # Apply undersampling on the training data
    sampling_strategy = 1 - sampling_ratio
    rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
    X_resampled, y_resampled = rus.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after undersampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100

    return X_train_resampled, y_resampled, zero_percentage_after


#SMOTE
def smote_data(X_train, y_train, sampling_ratio):

    # Set a custom sampling ratio
    sampling_ratio = 0.35

    # Apply SMOTE with the custom ratio
    smote = SMOTE(sampling_strategy=sampling_ratio, random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    # Reshape the resampled data
    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after oversampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100


    return X_train_resampled, y_resampled, zero_percentage_after


# ADASYN
def adasyn_oversample_data(X_train, y_train, sampling_ratio=0.35):

    # Apply ADASYN oversampling on the training data
    sampling_strategy = sampling_ratio
    adasyn = ADASYN(sampling_strategy=sampling_strategy, random_state=42, n_neighbors=5)
    X_resampled, y_resampled = adasyn.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    X_train_resampled = X_resampled.reshape(X_resampled.shape[0], X_train.shape[1], X_train.shape[2])

    # Calculate the percentage of 0s after oversampling
    zero_percentage_after = np.sum(y_resampled == 0) / len(y_resampled) * 100


    return X_train_resampled, y_resampled, zero_percentage_after


#Data sample mode
def sample_mode(X_train, y_train, sampling_ratio, oversample_mode):
 precentageOfZeroesInDataset = (y_train == 0).sum() / len(y_train)

 if oversample_mode == 'oversample':
      X_train, y_train, precentageOfZeroesInDataset = oversample_data(X_train, y_train, sampling_ratio)
 elif oversample_mode == 'undersample':
      X_train, y_train, precentageOfZeroesInDataset = undersample_data(X_train, y_train, sampling_ratio)
 elif oversample_mode == 'smote':
      X_train, y_train, precentageOfZeroesInDataset = smote_data(X_train, y_train, sampling_ratio)
 elif oversample_mode == 'adasyn':
      X_train, y_train, precentageOfZeroesInDataset = adasyn_oversample_data(X_train, y_train, sampling_ratio)


 return X_train, y_train, precentageOfZeroesInDataset

In [ ]:
#Confusion matrtix that shows injuries as TP
def confusion_matrix_only_injuries(y_test, y_pred):
    tp, fn, fp, tn = 0, 0, 0, 0
    for i, j in zip(y_test, y_pred):
        if i == 1 and j == 1:
            tp += 1
        elif i == 1 and j == 0:
            fn += 1
        elif i == 0 and j == 1:
            fp += 1
        elif i == 0 and j == 0:
            tn += 1
    # Create a DataFrame for the confusion matrix
    confusion_matrix = pd.DataFrame(
        {
            "Predicted Injuries": [tp, fp],
            "Predicted Non-Injuries": [fn, tn],
        },
        index=["Actual Injuries", "Actual Non-Injuries"]
    )

    return confusion_matrix

In [ ]:
# Datapreprocessing

def create_player_sequences(df, player, sequence_length=7):
    player_df = df[df['Player_name'] == player]
    # player_data = player_df.drop(['Date', 'Player_name'], axis=1).values
    player_data = player_df.drop(['Player_name'], axis=1).values

    sequences = []
    for i in range(len(player_data) - sequence_length):
        sequences.append(player_data[i:i + sequence_length])

    return np.array(sequences)

def create_team_sequence(df, sequence_length):
    players = df['Player_name'].unique()
    all_sequences = []
    for player in players:
        player_sequences = create_player_sequences(df, player, sequence_length)
        all_sequences.append(player_sequences)

    data = np.concatenate(all_sequences, axis=0)
    X, y = data[:, :-1, :], data[:, -1, -1]
    return X, y


def preprocess(inputWindow, interpolate=False):
    # Load your dataframe
    # df = merge_data_for_team("TeamA", interpolate=interpolate)
    df = pd.read_csv(dataPath)

    # # Convert 'Duration' column to seconds
    # df['Duration'] = df['Duration'].apply(convert_duration_to_seconds)

    # Preprocess the data
    X, y = create_team_sequence(df, inputWindow)

    # Convert data types
    X = X.astype(np.float32)
    y = y.astype(np.float32)
    return X, y


In [ ]:
%%capture
!pip install tslearn

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
from tslearn.preprocessing import TimeSeriesScalerMinMax
import numpy as np

def sample_mode(X_train, y_train, sampling_ratio, oversample_mode):
    # Placeholder for the actual sampling implementation
    return X_train, y_train, (y_train == 0).sum() / len(y_train) * 100

def confusion_matrix_only_injuries(y_test, y_pred):
    # Placeholder for the actual confusion matrix calculation
    return np.zeros((2, 2))

def decision_tree_classification(test_size, oversample_mode, sampling_ratio):
    # Preprocess the data
    X_scaled = TimeSeriesScalerMinMax().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size, random_state=42, stratify=y)
    precentageOfZeroesInDataset = (y_train == 0).sum() / len(y_train) * 100

    X_train, y_train, precentageOfZeroesInDataset = sample_mode(X_train, y_train, sampling_ratio, oversample_mode)

    # Reshape the data for Decision Tree
    X_train_2d = X_train.reshape(X_train.shape[0], -1)
    X_test_2d = X_test.reshape(X_test.shape[0], -1)

    # Define the hyperparameters grid
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt', 'log2']
    }

    # Initialize DecisionTreeClassifier
    clf = DecisionTreeClassifier()

    # Initialize GridSearchCV with F1 score as the scoring metric
    grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='f1')

    # Perform Grid Search Cross Validation
    grid_search.fit(X_train_2d, y_train)

    # Get the best parameters
    best_params = grid_search.best_params_

    # Get the best estimator
    best_clf = grid_search.best_estimator_

    # Evaluate the best model
    y_pred = best_clf.predict(X_test_2d)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    confInjuries = confusion_matrix_only_injuries(y_test, y_pred)

    return accuracy, f1, precision, recall, confInjuries, precentageOfZeroesInDataset


In [ ]:
#Single run
X, y = preprocess(7)
decision_tree_classification(0.1, 'adasyn', 0.3)

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


(0.983451536643026,
 0.0,
 0.0,
 0.0,
 array([[0., 0.],
        [0., 0.]]),
 99.35611038107753)

In [ ]:
#Multiple runs

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=UserWarning, message="`use_label_encoder` is deprecated in 1.7.0.")

# Create a list to store the results
results = []

# List of hyperparameters
input_windows =  [2, 4, 8, 16, 32]
test_sizes = [0.2, 0.3]
oversample_modes = ["none", "oversample", "undersample", "smote", "adasyn"]
sampling_ratios = [0.2, 0.3]
interploate_injuries = [True, False]



# Iterate through all combinations of hyperparameters
for interplate in interploate_injuries:
    for preprocess_param in input_windows:
        X, y = preprocess(preprocess_param)

        for test_size_param in test_sizes:
            for oversample_mode in oversample_modes:
                for sampling_ratio in sampling_ratios:

                    accuracy, f1, precision, recall, confusion, precentageOfZeroesInDataset = decision_tree_classification(test_size=test_size_param,
                                                                                    oversample_mode=oversample_mode,
                                                                                    sampling_ratio=sampling_ratio)

                    # Save the results
                    results.append({"input_windows": preprocess_param,
                                    "Interplate": interplate,
                                    "test_size": test_size_param,
                                    "oversample_mode": oversample_mode,
                                    "sampling_ratio": sampling_ratio,
                                    "Prectenage of zeroes in dataset": precentageOfZeroesInDataset,
                                    "accuracy": accuracy,
                                    "f1": f1,
                                    "precision": precision,
                                    "recall": recall,
                                    "confusion": confusion})

# Sort the results based on the F1 score
results.sort(key=lambda x: x['f1'], reverse=True)

# Print the top 3 performers
for i in range(min(3, len(results))):
    print(f"Top Performer {i + 1}:")
    print(f"Input window: {results[i]['input_windows']}")
    print(f"Interplate: {results[i]['Interplate']}")
    print(f"Test Size: {results[i]['test_size']}")
    print(f"Oversample Mode: {results[i]['oversample_mode']}")
    print(f"Sampling Ratio: {results[i]['sampling_ratio']}")
    print(f"Percentage of zeroes in dataset: {results[i]['Prectenage of zeroes in dataset']}")
    print(f"Accuracy: {results[i]['accuracy']}")
    print(f"F1: {results[i]['f1']}")
    print(f"Precision: {results[i]['precision']}")
    print(f"Recall: {results[i]['recall']}")
    display(results[i]['confusion'])
    print("\n")

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

Top Performer 1:
Input window: 32
Interplate: False
Test Size: 0.2
Oversample Mode: oversample
Sampling Ratio: 0.2
Percentage of zeroes in dataset: 99.35977514053717
Accuracy: 0.9943820224719101
F1: 0.47058823529411764
Precision: 0.5714285714285714
Recall: 0.4


array([[0., 0.],
       [0., 0.]])



Top Performer 2:
Input window: 32
Interplate: True
Test Size: 0.2
Oversample Mode: none
Sampling Ratio: 0.3
Percentage of zeroes in dataset: 99.35977514053717
Accuracy: 0.9906367041198502
F1: 0.34782608695652173
Precision: 0.3076923076923077
Recall: 0.4


array([[0., 0.],
       [0., 0.]])



Top Performer 3:
Input window: 32
Interplate: True
Test Size: 0.2
Oversample Mode: smote
Sampling Ratio: 0.3
Percentage of zeroes in dataset: 99.35977514053717
Accuracy: 0.9893882646691635
F1: 0.2608695652173913
Precision: 0.23076923076923078
Recall: 0.3


array([[0., 0.],
       [0., 0.]])